**In previous versions, I used different keras available models to see how they affect the results.**
Here is the complete list of <a href='https://keras.io/api/applications/'>[keras available models] </a> .

### Update: Training bigger networks (for example Xception in comparison with EfficientNetB0) with bigger image sizes and batch sizes will exhaust the GPU and result in an error. From version 5, I am using TPU to the train models. The reference notebook I used to learn is <a href='https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training'>[this great notebook] </a>

#### <span style="color:blue"> Version 12: using Xception, trainable = True, dropout 0.5. batch_size = 16*8, img_size = 900, random_flip_left_right, random_flip_up_down, random_saturation(0.8, 1.2), random_brightness(0.2), random_hue(0.2), random_contrast(0.8, 1.2), max_epoch = 30 with early stopping callback.</span>.

### This notebook is part 2 (for prediction) of the first notebook which was for training <a href='https://www.kaggle.com/sinamhd9/tensorflow-data-dataset-tpu-part1'>[Part 1] </a> Please take a look and upvote the <a href='https://www.kaggle.com/xhlulu/ranzcr-efficientnet-submission'>[reference notebook] </a>

In [ ]:
# Importing useful libraries

import pandas as pd 
import numpy as np
import os
import tensorflow as tf

In [ ]:
img_size = 900

In [ ]:
def auto_select_accelerator():
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy

def build_decoder(with_labels=True, target_size=(img_size, img_size), ext='jpg'):
    def decode(path):
        file_bytes = tf.io.read_file(path) # Reads and outputs the entire contents of the input filename.

        if ext == 'png':
            img = tf.image.decode_png(file_bytes, channels=3) # Decode a PNG-encoded image to a uint8 or uint16 tensor
        elif ext in ['jpg', 'jpeg']:
            img = tf.image.decode_jpeg(file_bytes, channels=3) # Decode a JPEG-encoded image to a uint8 tensor
        else:
            raise ValueError("Image extension not supported")

        img = tf.cast(img, tf.float32) / 255.0 # Casts a tensor to the type float32 and divides by 255.
        img = tf.image.resize(img, target_size) # Resizing to target size
        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_saturation(img, 0.8, 1.2)
        img = tf.image.random_brightness(img, 0.1)
        img = tf.image.random_contrast(img, 0.9, 1.2)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment

def build_dataset(paths, labels=None, bsize=32, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO) # overlaps data preprocessing and model execution while training
    return dset

In [ ]:
COMPETITION_NAME = "ranzcr-clip-catheter-line-classification"
strategy = auto_select_accelerator()
batch_size = strategy.num_replicas_in_sync * 16
print('batch size', batch_size)

In [ ]:
with strategy.scope():
    model = tf.keras.models.load_model('../input/tensorflow-data-dataset-tpu-part1/model.h5', custom_objects={'FixedDropout': tf.keras.layers.Dropout})

In [ ]:
model.summary()

In [ ]:
test_decoder = build_decoder(with_labels=False, target_size=(img_size, img_size))
load_dir = '/kaggle/input/ranzcr-clip-catheter-line-classification/'
df_sub = pd.read_csv(load_dir + 'sample_submission.csv')
test_paths = load_dir + "test/" + df_sub['StudyInstanceUID'] + '.jpg'
dtest = build_dataset(
    test_paths, bsize=batch_size, repeat=False, 
    shuffle=False, augment=False, cache=False, 
    decode_fn=test_decoder
)

In [ ]:
y_preds = model.predict(dtest, verbose=1)

In [ ]:
df_sub.iloc[:, 1:] = y_preds
display(df_sub)

In [ ]:
df_sub.to_csv('submission.csv',index=False)